In [1]:
import re

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (5,3)

In [3]:
PATH_FOLDER = "..\\data"

In [4]:
import oa_orders_dataclean
import oa_offers_dataclean

c:\users\benson\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (9,10,11,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
FILENAME_ZIPCODE_COORDINATES = "zipcode_coordinates.csv"
PATH_FILE_ZIPCODE_COORDINATES = os.path.join(PATH_FOLDER,"raw", FILENAME_ZIPCODE_COORDINATES)
zipcode_coordinates_orig = pd.read_csv(PATH_FILE_ZIPCODE_COORDINATES)
zipcode_coordinates = zipcode_coordinates_orig

In [6]:
def groupby_transform_value_count(df_column):
    column_name = list(df_column.columns)[0]
    temp_counter = df_column[column_name].value_counts()
    temp_counter = temp_counter.reset_index().rename(columns={"index":column_name,column_name:"count"})
    df_column = df_column.merge(temp_counter,on=[column_name])
    return df_column["count"]
def dataclean_ftl_duplicate_references(oa):
    '''
    Part of a greater data cleaning on removing incorrect FTL labels,
    Removes FTL rows that have reference numbers that show up more than once, as those rows are incorrectly labeled FTL
    '''
    assert "TRANSPORT_MODE" in oa.columns
    assert "REFERENCE_NUMBER" in oa.columns
    oa_ftl = oa[oa["TRANSPORT_MODE"]=="FTL"]
    temp_counter = oa_ftl["REFERENCE_NUMBER"].value_counts()
    temp_counter = temp_counter.reset_index().rename(columns={"index":"REFERENCE_NUMBER","REFERENCE_NUMBER":"count"})
    oa_ftl_counted = oa_ftl.merge(temp_counter,on=["REFERENCE_NUMBER"])
    oa_ftl_duplicate_reference_numbers = oa_ftl_counted[oa_ftl_counted["count"]>1]
    oa = oa.drop(list(oa_ftl_duplicate_reference_numbers.index), axis=0)    
    return oa

def dataclean_ftl_nonquote(oa):
    '''
    Part of a greater data cleaning on removing incorrect FTL labels,
    Removes FTL rows that don't have quote as the transport mode
    '''
    assert "OFFER_TYPE" in oa.columns
    assert "TRANSPORT_MODE" in oa.columns    
    oa = oa[~((oa["TRANSPORT_MODE"]=="FTL") & (oa["OFFER_TYPE"]!="quote"))]
    return oa


In [7]:
data_clean_first_time = 0

PATH_FILE_OA_OFFERS = os.path.join(PATH_FOLDER,"temp",oa_offers_dataclean.FILENAME_OA_OFFERS)

if data_clean_first_time:
    oa_offers = oa_offers_dataclean.oa_offers
    oa_offers = oa_offers_dataclean.dataclean(oa_offers)
    oa_offers.to_csv(PATH_FILE_OA_OFFERS,index=False)
else:
    oa_offers = pd.read_csv(PATH_FILE_OA_OFFERS)

In [8]:
data_clean_first_time = 0

PATH_FILE_OA_ORDERS = os.path.join(PATH_FOLDER,"temp",oa_orders_dataclean.FILENAME_OA_ORDERS)

if data_clean_first_time:
    oa_orders = oa_orders_dataclean.oa_orders
    oa_orders = oa_orders_dataclean.dataclean(oa_orders)
    oa_orders.to_csv(PATH_FILE_OA_ORDERS,index=False)
else:
    oa_orders = pd.read_csv(PATH_FILE_OA_ORDERS)

C:\Users\Benson\Desktop\2022_fall\dsc180a\data_oa\src\oa_orders_dataclean.py:60: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ).str.replace("[",""
C:\Users\Benson\Desktop\2022_fall\dsc180a\data_oa\src\oa_orders_dataclean.py:61: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ).str.replace("]",""


In [9]:
def dataclean():
    global oa_orders, oa_offers,zipcode_coordinates
    print("oa_orders.shape",oa_orders.shape)
    zipcode_coordinates["3DIGIT_ZIP"] =  zipcode_coordinates["3DIGIT_ZIP"].astype(str).str.zfill(3)
    oa_orders = oa_orders.merge(zipcode_coordinates, 
                                left_on=["ORIGIN_3DIGIT_ZIP"],
                                right_on=["3DIGIT_ZIP"],
                                 suffixes=(None, "_ORIG")
                               )

    oa_orders = oa_orders.merge(zipcode_coordinates, 
                                left_on=["DESTINATION_3DIGIT_ZIP"],
                                right_on=["3DIGIT_ZIP"],
                                 suffixes=(None, "_DEST")
                               )
    print("oa_orders.shape after geo join",oa_orders.shape)
    #oa_orders = dataclean_ftl_duplicate_references(oa_orders)

    oa = oa_orders.merge(oa_offers, on=["REFERENCE_NUMBER"],how="inner")
    print("oa.shape",oa.shape)
    oa = dataclean_ftl_nonquote(oa)
    print("oa.shape after cleaning ftl",oa.shape)
    return oa

In [10]:
data_clean_first_time = 0

FILENAME_OA = "oa.csv"

PATH_FILE_OA = os.path.join(PATH_FOLDER,"temp",FILENAME_OA)

if data_clean_first_time:
    oa = dataclean()
    oa.to_csv(PATH_FILE_OA,index=False)
else:
    oa = pd.read_csv(PATH_FILE_OA)

oa_orders.shape (280842, 24)
oa_orders.shape after geo join (280271, 30)
oa.shape (833321, 47)
oa.shape after cleaning ftl (833022, 47)
